In [1]:
import spacy
import gensim
import os
import re
import pandas as pd
from gensim.models import KeyedVectors
from joblib import Parallel, delayed
import itertools
import numpy as np
import pickle

In [2]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

data

In [3]:
%%time
dataset = pd.read_parquet('data/word_embeddings.parquet.gz')
dataset.head()

CPU times: user 38.2 s, sys: 15.4 s, total: 53.6 s
Wall time: 51.8 s


,media,url,timestamp,title,text,tokens,embeddings
0,sports-watch,http://news.livedoor.com/article/detail/5985384/,2011-11-02T09:00:00+0900,石川遼、“異例の交際発表”めぐる水面下の攻防とは？,先月24日深夜、プロゴルファー・石川遼が、マネジメント会社「ケーアイ企画」を通し、一般女性と...,"[先月, 24, 日, 深夜, 、, プロゴルファー, ・, 石川, 遼, が, 、, マネ...","[[-0.071045354, -0.25821245, 0.14480348, -0.22..."
1,sports-watch,http://news.livedoor.com/article/detail/6819620/,2012-08-03T10:00:00+0900,柔道界の現状に、古賀氏「日本の場合は先生の圧力が強い」,ロンドン五輪における柔道男子は、ここまで金メダルを獲得できずにいる。2日も男子100kg級に...,"[ロンドン, 五輪, に, おけ, る, 柔道, 男子, は, 、, ここ, まで, 金, ...","[[-0.09269952, 0.5426098, 0.12492261, -0.18234..."
2,sports-watch,http://news.livedoor.com/article/detail/5571206/,2011-05-20T02:00:00+0900,【Sports Watch】女性店員の契約選手侮辱ツイートでアディダスが謝罪,ヴァンフォーレ甲府のFWで、アディダス契約選手の一人＝ハーフナー・マイクが、18日に入籍を発...,"[ヴァン, フォーレ, 甲府, の, FW, で, 、, アディダス, 契約, 選手, の,...","[[-0.12361497, 0.050540913, -0.46253157, -0.16..."
3,sports-watch,http://news.livedoor.com/article/detail/4970500/,2010-08-27T07:25:00+0900,【Sports Watch】スエマエの五輪後、前田は「人間不信になった感じ」,2008年、北京五輪の舞台で世界ランキング一位のペアから金星を挙げ、一躍その名を知らしめた女...,"[2008, 年, 、, 北京, 五輪, の, 舞台, で, 世界, ランキング, 一, 位...","[[0.19916902, -0.079966575, 0.2989788, -0.0169..."
4,sports-watch,http://news.livedoor.com/article/detail/6158040/,2011-12-30T09:00:00+0900,天皇杯で敗れたチームのファン（？）、チケットを燃やす,29日に行なわれた第91回天皇杯全日本サッカー選手権大会は、J2のFC東京と京都サンガが決勝...,"[29, 日, に, 行なわ, れ, た, 第, 91, 回, 天皇, 杯, 全, 日本, ...","[[0.19836882, -0.106275775, 0.3315136, -0.2118..."


## prepare dataset

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [5]:
# tmp
dataset.loc[:, 'embeddings'] = dataset['embeddings'].apply(lambda x: np.array(list(x)))
dataset.head()

,media,url,timestamp,title,text,tokens,embeddings
0,sports-watch,http://news.livedoor.com/article/detail/5985384/,2011-11-02T09:00:00+0900,石川遼、“異例の交際発表”めぐる水面下の攻防とは？,先月24日深夜、プロゴルファー・石川遼が、マネジメント会社「ケーアイ企画」を通し、一般女性と...,"[先月, 24, 日, 深夜, 、, プロゴルファー, ・, 石川, 遼, が, 、, マネ...","[[-0.071045354, -0.25821245, 0.14480348, -0.22..."
1,sports-watch,http://news.livedoor.com/article/detail/6819620/,2012-08-03T10:00:00+0900,柔道界の現状に、古賀氏「日本の場合は先生の圧力が強い」,ロンドン五輪における柔道男子は、ここまで金メダルを獲得できずにいる。2日も男子100kg級に...,"[ロンドン, 五輪, に, おけ, る, 柔道, 男子, は, 、, ここ, まで, 金, ...","[[-0.09269952, 0.5426098, 0.12492261, -0.18234..."
2,sports-watch,http://news.livedoor.com/article/detail/5571206/,2011-05-20T02:00:00+0900,【Sports Watch】女性店員の契約選手侮辱ツイートでアディダスが謝罪,ヴァンフォーレ甲府のFWで、アディダス契約選手の一人＝ハーフナー・マイクが、18日に入籍を発...,"[ヴァン, フォーレ, 甲府, の, FW, で, 、, アディダス, 契約, 選手, の,...","[[-0.12361497, 0.050540913, -0.46253157, -0.16..."
3,sports-watch,http://news.livedoor.com/article/detail/4970500/,2010-08-27T07:25:00+0900,【Sports Watch】スエマエの五輪後、前田は「人間不信になった感じ」,2008年、北京五輪の舞台で世界ランキング一位のペアから金星を挙げ、一躍その名を知らしめた女...,"[2008, 年, 、, 北京, 五輪, の, 舞台, で, 世界, ランキング, 一, 位...","[[0.19916902, -0.079966575, 0.2989788, -0.0169..."
4,sports-watch,http://news.livedoor.com/article/detail/6158040/,2011-12-30T09:00:00+0900,天皇杯で敗れたチームのファン（？）、チケットを燃やす,29日に行なわれた第91回天皇杯全日本サッカー選手権大会は、J2のFC東京と京都サンガが決勝...,"[29, 日, に, 行なわ, れ, た, 第, 91, 回, 天皇, 杯, 全, 日本, ...","[[0.19836882, -0.106275775, 0.3315136, -0.2118..."


In [6]:
X = np.array(dataset['embeddings'].to_list())
y = dataset['media']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=dataset['media'], random_state=5108)

In [7]:
# check if stratified
y_train_ratio = y_train.value_counts()/y_train.value_counts().sum()
y_test_ratio  = y_test.value_counts()/y_test.value_counts().sum()
ratio = pd.concat([y_train_ratio, y_test_ratio], axis=1)
ratio.columns = ['train', 'test']
ratio.style.background_gradient('Blues')

,train,test
sports-watch,0.122194,0.122150
movie-enter,0.118030,0.118350
it-life-hack,0.118030,0.118350
smax,0.118030,0.118350
dokujo-tsushin,0.118030,0.117807
kaden-channel,0.117306,0.117264
peachy,0.114410,0.114007
topic-news,0.104634,0.104235
livedoor-homme,0.069334,0.069490


## modeling

In [8]:
import torch
import torch.nn as nn

In [9]:
dataset.loc[:, 'embeddings_tensor'] = dataset['embeddings'].apply(lambda x: torch.tensor(np.array(list(x))))
dataset.head()
print(dataset['embeddings_tensor'][0])

tensor([[-0.0710, -0.2582,  0.1448,  ...,  0.0241, -0.0913, -0.0457],
        [ 0.2202, -0.0901,  0.2460,  ...,  0.0837, -0.1585, -0.1565],
        [ 0.1135, -0.1536,  0.2635,  ..., -0.0146, -0.0152, -0.0576],
        ...,
        [-0.2400,  0.4199,  0.0664,  ...,  0.6300,  0.0596,  0.0113],
        [ 0.1291, -0.0587,  0.2667,  ..., -0.0918, -0.0228, -0.2439],
        [ 0.1614, -0.0502,  0.2891,  ..., -0.1078, -0.0388, -0.2318]])


(`単語数，文数，次元数)

In [10]:
# VOCAB_SIZE = len(word2index)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
BATCH_SIZE = 1

emb = dataset['embeddings_tensor'][0]
lstm_input = emb.view(len(emb), BATCH_SIZE, -1)
lstm_input.shape

torch.Size([490, 1, 300])

In [11]:
lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)

out1, out2 = lstm(lstm_input)
print(out1.shape)
print(out2)

torch.Size([490, 1, 128])
(tensor([[[ 0.0026,  0.0360, -0.0038,  0.1203,  0.0445, -0.0641, -0.0641,
          -0.1231, -0.0287, -0.0383,  0.0543, -0.0459, -0.0544,  0.0604,
           0.1016, -0.0589, -0.0586, -0.0043, -0.0181,  0.1761,  0.1033,
           0.0841, -0.0822, -0.1888, -0.0917, -0.0341,  0.0804, -0.1273,
          -0.0764,  0.0448,  0.0290, -0.0557, -0.0136,  0.0372,  0.0777,
           0.0141, -0.0853, -0.0288,  0.0588,  0.0274,  0.0863,  0.0801,
           0.1012,  0.1111, -0.1675,  0.0187,  0.0510,  0.0210,  0.1048,
          -0.0294, -0.0236,  0.0527, -0.1596, -0.0649, -0.1114,  0.0475,
           0.1013,  0.0400,  0.0127, -0.0958, -0.0526, -0.1274,  0.1216,
          -0.0366,  0.0344,  0.0907, -0.1361, -0.0632,  0.0155,  0.0591,
           0.1122,  0.0287, -0.1940, -0.0412,  0.1094, -0.0600, -0.0131,
          -0.0745,  0.0422,  0.0314,  0.0116,  0.0019, -0.0383, -0.0052,
           0.0842,  0.1230,  0.0211, -0.0063, -0.0174, -0.0632, -0.1162,
           0.0142, -0.02

In [12]:
class LSTMClassifier(nn.Module):
    # モデルで使う各ネットワークをコンストラクタで定義
    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        self.softmax = nn.LogSoftmax(dim=1)

    # 順伝播処理はforward関数に記載
    def forward(self, sentence):
#         # 文章内の各単語をベクトル化して出力。2次元のテンソル
#         embeds = self.word_embeddings(sentence)
        # 2次元テンソルをLSTMに食わせられる様にviewで３次元テンソルにした上でLSTMへ流す。
        # 上記で説明した様にmany to oneのタスクを解きたいので、第二戻り値だけ使う。
        embeds = torch.tensor(list(sentence))
        _, lstm_out = self.lstm(embeds.view(len(sentence), 1, -1))
        # lstm_out[0]は３次元テンソルになってしまっているので2次元に調整して全結合。
        tag_space = self.hidden2tag(lstm_out[0].view(-1, self.hidden_dim))
        # softmaxに食わせて、確率として表現
        tag_scores = self.softmax(tag_space)
        return tag_scores

EMBEDDING_DIM = 300
HIDDEN_DIM = 128
BATCH_SIZE = 1
TARGET_SIZE = len(dataset['media'].unique())

model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, TARGET_SIZE)

In [13]:
model

LSTMClassifier(
  (lstm): LSTM(300, 128)
  (hidden2tag): Linear(in_features=128, out_features=9, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [14]:
categories = list(dataset['media'].unique())
category2index = {}
for cat in categories:
    if cat in category2index: continue
    category2index[cat] = len(category2index)
print(category2index)
#{'movie-enter': 0, 'it-life-hack': 1, 'kaden-channel': 2, 'topic-news': 3, 'livedoor-homme': 4, 'peachy': 5, 'sports-watch': 6, 'dokujo-tsushin': 7, 'smax': 8}

def category2tensor(cat):
    return torch.tensor([category2index[cat]], dtype=torch.long)

print(category2tensor("it-life-hack"))
# tensor([1])

{'sports-watch': 0, 'peachy': 1, 'smax': 2, 'livedoor-homme': 3, 'movie-enter': 4, 'kaden-channel': 5, 'it-life-hack': 6, 'dokujo-tsushin': 7, 'topic-news': 8}
tensor([6])


In [15]:
X_train[0]

array([[ 0.27850372, -0.14280288,  0.0590597 , ..., -0.13369516,
        -0.15940286, -0.08973489],
       [ 0.11345748, -0.1536053 ,  0.26349765, ..., -0.01463737,
        -0.01519805, -0.05762682],
       [ 0.15083103, -0.13699119, -0.17893592, ..., -0.0339943 ,
         0.02119296, -0.14708784],
       ...,
       [-0.00744801, -0.16674292,  0.1395167 , ...,  0.10811774,
         0.1102052 ,  0.04362018],
       [ 0.23267491, -0.0988275 , -0.08964849, ..., -0.00348453,
         0.0086258 , -0.16862108],
       [-0.03235267, -0.24351938,  0.01377656, ..., -0.00451565,
        -0.0897551 , -0.23788233]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
import torch.optim as optim

# 単語のベクトル次元数
EMBEDDING_DIM = 300
# 隠れ層の次元数
HIDDEN_DIM = 128
# # データ全体の単語数
# VOCAB_SIZE = len(word2index)
# 分類先のカテゴリの数
TAG_SIZE = len(categories)
# モデル宣言
model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, TAG_SIZE)
# 損失関数はNLLLoss()を使う。LogSoftmaxを使う時はこれを使うらしい。
loss_function = nn.NLLLoss()
# 最適化の手法はSGDで。lossの減りに時間かかるけど、一旦はこれを使う。
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 各エポックの合計loss値を格納する
losses = []
# 100ループ回してみる。（バッチ化とかGPU使ってないので結構時間かかる...）
for epoch in range(100):
    all_loss = 0
    for e, (embedding, cat) in enumerate(zip(X_train, y_train)):
        # モデルが持ってる勾配の情報をリセット
        model.zero_grad()
        # 順伝播の結果を受け取る
        out = model(embedding)
        # 正解カテゴリをテンソル化
        answer = category2tensor(cat)
        # 正解とのlossを計算
        loss = loss_function(out, answer)
        # 勾配をセット
        loss.backward()
        # 逆伝播でパラメータ更新
        optimizer.step()
        # lossを集計
        all_loss += loss.item()
        if (e%1000)==0: print(epoch, '-', e)
    losses.append(all_loss)
    print("epoch", epoch, "\t" , "loss", all_loss)
print("done.")

0 - 0


/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


0 - 1000
0 - 2000
0 - 3000
0 - 4000
0 - 5000
epoch 0 	 loss 9011.948988019489
1 - 0
1 - 1000
1 - 2000
1 - 3000
1 - 4000
1 - 5000
epoch 1 	 loss 5637.201208098704
2 - 0
2 - 1000
2 - 2000
2 - 3000
2 - 4000
2 - 5000
epoch 2 	 loss 4935.312842182102
3 - 0
3 - 1000
3 - 2000
3 - 3000
3 - 4000
3 - 5000
epoch 3 	 loss 4611.952290912566
4 - 0
4 - 1000
4 - 2000
4 - 3000
4 - 4000
4 - 5000
epoch 4 	 loss 4137.66166933428
5 - 0
5 - 1000
5 - 2000
5 - 3000
5 - 4000
5 - 5000
epoch 5 	 loss 4333.362538182468
6 - 0
6 - 1000
6 - 2000
6 - 3000
6 - 4000
6 - 5000
epoch 6 	 loss 4047.497596488858
7 - 0
7 - 1000
7 - 2000
7 - 3000
7 - 4000
7 - 5000
epoch 7 	 loss 2943.3440656845123
8 - 0
8 - 1000
8 - 2000
8 - 3000
8 - 4000
8 - 5000
epoch 8 	 loss 2412.965718171181
9 - 0
9 - 1000
9 - 2000
9 - 3000
9 - 4000


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(losses)

In [ ]:
# テストデータの母数計算
test_num = len(testdata)
# 正解の件数
a = 0
# 勾配自動計算OFF
with torch.no_grad():
    for embeddig, category in zip(X_test, y_test):
#         # テストデータの予測
#         inputs = sentence2index(title)
        out = model(embeddig)

        # outの一番大きい要素を予測結果をする
        _, predict = torch.max(out, 1)

        answer = category2tensor(category)
        if predict == answer:
            a += 1
print("predict : ", a / test_num)
# predict :  0.6118391323994578